In [7]:
import requests
import json

from itertools import combinations

import pandas as pd

from amp import api_handler

In [58]:
def build_taxonomy_reference(companies, country):
    tax = pd.DataFrame()
    
    #countries = [country['code'] for country in api.getAllCountries()]
    for company in companies:
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json',
            'X-AUTH-TOKEN': api.jwt,
        }

        data = {"companyId": company, "countryCode": country,  "taxonomyFilter": "ALL", "omitCombination": False}
        data = json.dumps(data)
        r = requests.post('https://amp.adsquare.com/api/v1/audience360/segments/list', headers=headers, data=data)
        tmp = pd.DataFrame(r.json())
        tax = tax.append(tmp)
        
    return tax[(tax['attentionRequiredCodes'].apply(lambda x: str(x)) == '[0]') & (tax.deletable==False) & (tax.type == 'SEGMENT')]

In [18]:
def retr_search_space(blocklist=[]):
    api = api_handler.AdsquareAmpApi()
    companies = api.getAllCompanies()
    dataproviders = []
    for company in companies:
        if company['dataprovider'] == True:
            dataproviders.append(company)
    companyIds = [provider['id'] for provider in dataproviders]
    for blocked in blocklist:
        companyIds.remove(blocked)
    return companyIds    

In [71]:
def get_simple_union(segments):
    counter = 1
    union_segments = []

    for segment in segments:
        union_segments.append({"segmentId": segment, "type": "SEGMENT", "active": True, "order": counter})
        counter += 1

    query = {"combination": {"order": 1, "combinations": [
        {"order": 1, "combinations": [], "segments": union_segments, "operator": "OR"}
                                                         ], "segments": [], "operator": "OR"}}
    data = json.dumps(query)           


    r = api.post('https://amp.adsquare.com/api/v1/audience360/segments/calculateReach',
                          headers=api.headers, data=data)

    return r.json()['maxUniqueUsers']

def get_simple_intersection(segments):
    counter = 1
    intersecting_segments = []

    for segment in segments:
        intersecting_segments.append({"segmentId": segment, "type": "SEGMENT", "active": True, "order": counter})
        counter += 1

    query = {"combination": {"order": 1, "combinations": [
        {"order": 1, "combinations": [], "segments": intersecting_segments, "operator": "AND"}
                                                         ], "segments": [], "operator": "OR"}}
    data = json.dumps(query)           


    r = api.post('https://amp.adsquare.com/api/v1/audience360/segments/calculateReach',
                          headers=api.headers, data=data)

    return r.json()['maxUniqueUsers']

def get_reach(self, segments):

    counter = 1
    intersecting_segments = []

    for segment in segments:
        intersecting_segments.append({"segmentId": segment, "type": "SEGMENT", "active": True, "order": counter})
        counter += 1

    query = {"combination": {"order": 1, "combinations": [
        {"order": 1, "combinations": [], "segments": intersecting_segments, "operator": "OR"}
                                                         ], "segments": [], "operator": "OR"}}
    data = dumps(query)           


    r = self.api.post('https://amp.adsquare.com/api/v1/audience360/segments/calculateReach',
                          headers=api.headers, data=data)

    return r.json()['maxUniqueUsers']

In [19]:
companies = retr_search_space(blocklist=['57c9a510e4b0fa139026c7af'])

In [59]:
tax = build_taxonomy_reference(companies, 'DE')

In [63]:
tax.reset_index(drop=True, inplace=True)

In [69]:
get_union(['2472435', '2585891'])

10075032

In [70]:
get_union(['2585891'])

9751312

In [ ]:
target = ['2585891']

In [83]:
segments = list(combinations(tax.id,2))

In [82]:
len(tax.id)*len(tax.id) - len(tax.id)

6162806

In [85]:
segments

[(2477838, 2477531),
 (2477838, 2477527),
 (2477838, 2477812),
 (2477838, 2477300),
 (2477838, 2477419),
 (2477838, 2477997),
 (2477838, 2478246),
 (2477838, 2477762),
 (2477838, 2477664),
 (2477838, 2477247),
 (2477838, 2477522),
 (2477838, 2477469),
 (2477838, 2477558),
 (2477838, 2477735),
 (2477838, 2477276),
 (2477838, 2477835),
 (2477838, 2477723),
 (2477838, 2477555),
 (2477838, 2477609),
 (2477838, 2477499),
 (2477838, 2477754),
 (2477838, 2477831),
 (2477838, 2478137),
 (2477838, 2477728),
 (2477838, 2477774),
 (2477838, 2478013),
 (2477838, 2477986),
 (2477838, 2477984),
 (2477838, 2477814),
 (2477838, 2477718),
 (2477838, 2478206),
 (2477838, 2477729),
 (2477838, 2478024),
 (2477838, 2477836),
 (2477838, 2478076),
 (2477838, 2478010),
 (2477838, 2478031),
 (2477838, 2478026),
 (2477838, 2478406),
 (2477838, 2477731),
 (2477838, 2477733),
 (2477838, 2477830),
 (2477838, 2477599),
 (2477838, 2478015),
 (2477838, 2478143),
 (2477838, 2477656),
 (2477838, 2477746),
 (2477838, 24

In [88]:
import random
from numpy.random import choice


In [106]:
sets = dict()
sets[1] = set()
sets[2] = set()
sets[3] = set()
sets[4] = set()
sets[5] = set()

In [107]:
for i in range(1000000):
    conds = choice([1,2,3,4,5], size=random.randint(1,5), p=[.26,.24,.1,.35,.05])
    for cond in conds:
        sets[cond].add(i)

In [108]:
for i in [1,2,3,4,5]:
    print(i,len(sets[i]))

(1, 556453)
(2, 527077)
(3, 263125)
(4, 671379)
(5, 140072)


In [111]:
target = choice(list(range(1000000)), size=25000)

In [114]:
target = set(target)

In [115]:
len(target)

24671

In [116]:
for i in [1,2,3,4,5]:
    print(i, len(target.intersection(sets[i])))

(1, 13785)
(2, 12859)
(3, 6570)
(4, 16479)
(5, 3494)


In [122]:
print('Weighted intersection probability of A belongs Target')
for i in [1,2,3,4,5]:
    print(i, len(target.intersection(sets[i]))/float(len(sets[i])))

Weighted intersection probability of A belongs Target
(1, 0.024772981725320917)
(2, 0.024396814886629467)
(3, 0.02496912114014252)
(4, 0.024545003641758233)
(5, 0.02494431435261865)


In [120]:
print('Weighted set similarity | 0 >= x <= 1')
for i in [1,2,3,4,5]:
    print(i, len(target.intersection(sets[i]))/float(len(target.union(sets[i]))))

Weighted set similarity | 0 >= x <= 1
(1, 0.024297642150460305)
(2, 0.023862056935658363)
(3, 0.023361993556783513)
(4, 0.02424912187247543)
(5, 0.021668351431636784)


In [121]:
len(target.intersection(sets[i])), len(target.union(sets[i]))

(3494, 161249)

In [123]:
len(target.intersection(sets[4])), len(target.union(sets[4]))

(16479, 679571)

In [124]:
0.024772981725320917*float(len(sets[i]))

3470.0010962291517

In [126]:
len(sets[i]), len(target)

(140072, 24671)